In [56]:
import warnings
warnings.filterwarnings("ignore")
#Working on plotly
import chart_studio.plotly as py
import cufflinks as cf
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode,plot,iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
cf.go_offline()
import pandas as pd
from datetime import datetime

from scipy.stats import kurtosis
from statsmodels.tsa.api import ExponentialSmoothing,SimpleExpSmoothing,Holt
import numpy as np
from numpy.fft import fft
import matplotlib.pyplot as plt
%matplotlib inline
import os

In [57]:
df=pd.read_csv(r"..\ShurovoData_2018H2\ShurovoData_2018H2\RAW3_1.txt")
df.head()

,Number of points: 8192
0,Sampling rate(Hz): 3125
1,Time range: From: 07/01/2018 08:00:12; Until: ...
2,Units: g
3,01.07.2018 08:00:12.7250048\t-0.1958597
4,01.07.2018 08:00:12.7253248\t0.1991889


In [58]:
fs=3125
df.columns=['RAW3_1']
df = pd.DataFrame(df['RAW3_1'].str.split('\t',2).tolist())

In [59]:
df.dropna(inplace=True)

In [60]:
df

,0,1
3,01.07.2018 08:00:12.7250048,-0.1958597
4,01.07.2018 08:00:12.7253248,0.1991889
5,01.07.2018 08:00:12.7256448,-0.1089342
6,01.07.2018 08:00:12.7259648,0.09857726
7,01.07.2018 08:00:12.7262848,0.09820724
...,...,...
1895056,01.12.2018 18:00:02.6298368,0.9375005
1895057,01.12.2018 18:00:02.6301568,0.5635362
1895058,01.12.2018 18:00:02.6304768,0.01313138
1895059,01.12.2018 18:00:02.6307968,0.3556547


In [61]:
df=df[:5000]

In [62]:
df[0]=pd.to_datetime(df[0])
df["Date_Time"]=pd.to_datetime(df[0])
df['Seconds'] = df["Date_Time"].apply(lambda x:int(x.strftime('%Y-%m-%d %H:%M:%S.%f')[20:])/1000000)
df

,0,1,Date_Time,Seconds
3,2018-01-07 08:00:12.725004,-0.1958597,2018-01-07 08:00:12.725004,0.725004
4,2018-01-07 08:00:12.725324,0.1991889,2018-01-07 08:00:12.725324,0.725324
5,2018-01-07 08:00:12.725644,-0.1089342,2018-01-07 08:00:12.725644,0.725644
6,2018-01-07 08:00:12.725964,0.09857726,2018-01-07 08:00:12.725964,0.725964
7,2018-01-07 08:00:12.726284,0.09820724,2018-01-07 08:00:12.726284,0.726284
...,...,...,...,...
4998,2018-01-07 08:00:14.323404,0.4629245,2018-01-07 08:00:14.323404,0.323404
4999,2018-01-07 08:00:14.323724,-0.6212397,2018-01-07 08:00:14.323724,0.323724
5000,2018-01-07 08:00:14.324044,-0.3630526,2018-01-07 08:00:14.324044,0.324044
5001,2018-01-07 08:00:14.324364,0.280196,2018-01-07 08:00:14.324364,0.324364


In [65]:
df.Seconds

3       0.725004
4       0.725324
5       0.725644
6       0.725964
7       0.726284
          ...   
4998    0.323404
4999    0.323724
5000    0.324044
5001    0.324364
5002    0.324684
Name: Seconds, Length: 5000, dtype: float64

In [ ]:
#df['seconds'] = df.seconds.apply(lambda x:datetime.strptime(x.strftime("%Y-%m-%d %H:%M:%S"),"%Y-%m-%d %H:%M:%S"))

In [66]:
df[0]=df[0].apply(lambda x:datetime.strptime(x.strftime("%Y-%m-%d %H:%M:%S"),"%Y-%m-%d %H:%M:%S"))

In [67]:
df.index=df[0]

In [68]:
df['date']=df.index.date
df['time']=df.index.time

In [69]:
df

,0,1,Date_Time,Seconds,date,time
0,,,,,,
2018-01-07 08:00:12,2018-01-07 08:00:12,-0.1958597,2018-01-07 08:00:12.725004,0.725004,2018-01-07,08:00:12
2018-01-07 08:00:12,2018-01-07 08:00:12,0.1991889,2018-01-07 08:00:12.725324,0.725324,2018-01-07,08:00:12
2018-01-07 08:00:12,2018-01-07 08:00:12,-0.1089342,2018-01-07 08:00:12.725644,0.725644,2018-01-07,08:00:12
2018-01-07 08:00:12,2018-01-07 08:00:12,0.09857726,2018-01-07 08:00:12.725964,0.725964,2018-01-07,08:00:12
2018-01-07 08:00:12,2018-01-07 08:00:12,0.09820724,2018-01-07 08:00:12.726284,0.726284,2018-01-07,08:00:12
...,...,...,...,...,...,...
2018-01-07 08:00:14,2018-01-07 08:00:14,0.4629245,2018-01-07 08:00:14.323404,0.323404,2018-01-07,08:00:14
2018-01-07 08:00:14,2018-01-07 08:00:14,-0.6212397,2018-01-07 08:00:14.323724,0.323724,2018-01-07,08:00:14
2018-01-07 08:00:14,2018-01-07 08:00:14,-0.3630526,2018-01-07 08:00:14.324044,0.324044,2018-01-07,08:00:14


In [70]:
df.drop(columns=[0],inplace=True)

In [71]:
df

,1,Date_Time,Seconds,date,time
0,,,,,
2018-01-07 08:00:12,-0.1958597,2018-01-07 08:00:12.725004,0.725004,2018-01-07,08:00:12
2018-01-07 08:00:12,0.1991889,2018-01-07 08:00:12.725324,0.725324,2018-01-07,08:00:12
2018-01-07 08:00:12,-0.1089342,2018-01-07 08:00:12.725644,0.725644,2018-01-07,08:00:12
2018-01-07 08:00:12,0.09857726,2018-01-07 08:00:12.725964,0.725964,2018-01-07,08:00:12
2018-01-07 08:00:12,0.09820724,2018-01-07 08:00:12.726284,0.726284,2018-01-07,08:00:12
...,...,...,...,...,...
2018-01-07 08:00:14,0.4629245,2018-01-07 08:00:14.323404,0.323404,2018-01-07,08:00:14
2018-01-07 08:00:14,-0.6212397,2018-01-07 08:00:14.323724,0.323724,2018-01-07,08:00:14
2018-01-07 08:00:14,-0.3630526,2018-01-07 08:00:14.324044,0.324044,2018-01-07,08:00:14


In [75]:
df.index=df.index.astype('str')

In [76]:
new_df=df.loc['2018-01-07 08:00:13']

In [78]:
new_df

,1,Date_Time,Seconds,date,time
0,,,,,
2018-01-07 08:00:13,-0.1285386,2018-01-07 08:00:13.000204,0.000204,2018-01-07,08:00:13
2018-01-07 08:00:13,0.2195332,2018-01-07 08:00:13.000524,0.000524,2018-01-07,08:00:13
2018-01-07 08:00:13,-0.1392655,2018-01-07 08:00:13.000844,0.000844,2018-01-07,08:00:13
2018-01-07 08:00:13,-0.003883839,2018-01-07 08:00:13.001164,0.001164,2018-01-07,08:00:13
2018-01-07 08:00:13,0.08082223,2018-01-07 08:00:13.001484,0.001484,2018-01-07,08:00:13
...,...,...,...,...,...
2018-01-07 08:00:13,0.2668798,2018-01-07 08:00:13.998604,0.998604,2018-01-07,08:00:13
2018-01-07 08:00:13,0.2176836,2018-01-07 08:00:13.998924,0.998924,2018-01-07,08:00:13
2018-01-07 08:00:13,-0.5864694,2018-01-07 08:00:13.999244,0.999244,2018-01-07,08:00:13


In [79]:
new_df.columns=['y','Date_time','Seconds','date','time']

In [80]:
new_df

,y,Date_time,Seconds,date,time
0,,,,,
2018-01-07 08:00:13,-0.1285386,2018-01-07 08:00:13.000204,0.000204,2018-01-07,08:00:13
2018-01-07 08:00:13,0.2195332,2018-01-07 08:00:13.000524,0.000524,2018-01-07,08:00:13
2018-01-07 08:00:13,-0.1392655,2018-01-07 08:00:13.000844,0.000844,2018-01-07,08:00:13
2018-01-07 08:00:13,-0.003883839,2018-01-07 08:00:13.001164,0.001164,2018-01-07,08:00:13
2018-01-07 08:00:13,0.08082223,2018-01-07 08:00:13.001484,0.001484,2018-01-07,08:00:13
...,...,...,...,...,...
2018-01-07 08:00:13,0.2668798,2018-01-07 08:00:13.998604,0.998604,2018-01-07,08:00:13
2018-01-07 08:00:13,0.2176836,2018-01-07 08:00:13.998924,0.998924,2018-01-07,08:00:13
2018-01-07 08:00:13,-0.5864694,2018-01-07 08:00:13.999244,0.999244,2018-01-07,08:00:13


In [81]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3125 entries, 2018-01-07 08:00:13 to 2018-01-07 08:00:13
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   y          3125 non-null   object        
 1   Date_time  3125 non-null   datetime64[ns]
 2   Seconds    3125 non-null   float64       
 3   date       3125 non-null   object        
 4   time       3125 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 146.5+ KB


In [82]:
new_df.y=new_df.y.astype('float')

In [83]:
len(new_df.y)

3125

In [84]:
new_df['Date_time'] = pd.to_datetime(new_df['Date_time'])
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3125 entries, 2018-01-07 08:00:13 to 2018-01-07 08:00:13
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   y          3125 non-null   float64       
 1   Date_time  3125 non-null   datetime64[ns]
 2   Seconds    3125 non-null   float64       
 3   date       3125 non-null   object        
 4   time       3125 non-null   object        
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 146.5+ KB


In [85]:
new_df['Date_time']

0
2018-01-07 08:00:13   2018-01-07 08:00:13.000204
2018-01-07 08:00:13   2018-01-07 08:00:13.000524
2018-01-07 08:00:13   2018-01-07 08:00:13.000844
2018-01-07 08:00:13   2018-01-07 08:00:13.001164
2018-01-07 08:00:13   2018-01-07 08:00:13.001484
                                 ...            
2018-01-07 08:00:13   2018-01-07 08:00:13.998604
2018-01-07 08:00:13   2018-01-07 08:00:13.998924
2018-01-07 08:00:13   2018-01-07 08:00:13.999244
2018-01-07 08:00:13   2018-01-07 08:00:13.999564
2018-01-07 08:00:13   2018-01-07 08:00:13.999884
Name: Date_time, Length: 3125, dtype: datetime64[ns]

In [87]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=new_df['Seconds'],y=new_df.y,mode='lines',name = 'time signal'))
fig.update_layout(hovermode="x")
fig.update_layout(xaxis=dict(showgrid=True),yaxis=dict(showgrid=True),title={'text':'Time signal','xanchor':'center','x':0.5})
fig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Amplitude')

In [88]:
N=len(new_df)

In [89]:
N

3125

In [90]:
fs=3125

In [91]:
fstep=int(fs/N)

In [92]:
f=np.linspace(0,(N-1)*fstep,N)

In [93]:
X=np.fft.fft(new_df['y'])

In [94]:
X

array([138.58548521+0.j        ,  -0.30930309+2.74206046j,
         0.81614909-1.54820957j, ...,  -2.42153862-2.17375409j,
         0.81614909+1.54820957j,  -0.30930309-2.74206046j])

In [95]:
X_mag=np.abs((X))/N

In [96]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=f[:int(len(X_mag)//2)],y=X_mag[:int(len(X_mag)//2)],mode='lines',name = 'fft'))
fig.update_layout(hovermode="x")
fig.update_layout(xaxis=dict(showgrid=True),yaxis=dict(showgrid=True),title={'text':'FFT','xanchor':'center','x':0.5})
fig.update_xaxes(title_text='Frequency (HZ)')
fig.update_yaxes(title_text='Amplitude')